# Observer: property observers

> Merging Python properties with the Observer design pattern

You might remember than in Python, a **property** is a special kind of attribute with `__get__`, `__set__` and/or `__delete__` methods.

A **property observer** tells us whenever a property is actually changed.

We will now create a `PropertyObservable` class that other classes can inherit from in order to offer observable properties. We will then create a `Person` class with an `age` property that will trigger an event whenever it gets changed; we can do this from its setter:

In [1]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

class PropertyObservable:
    def __init__(self):
        self.property_changed = Event()

class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        if self._age == value: # we're not actually changing the value, so there's nothing to modify or notify
            return
        self._age = value
        self.property_changed('age', value) # the value changes, so we trigger an event with 2 arguments: the property name and its value

Our observable property is ready; we just need to subscribe to it now. We will create a `TrafficAuthority` class that is interested in monitoring whether a `Person` is old enough to drive, so it will subscribe to that person's `age` property:

In [ ]:
class TrafficAuthority:
    def __init__(self, person):
        self.person = person
        person.property_changed.append(self.person_changed) # subscribing to self.person's events with person_changed

    def person_changed(self, name, value):
        if name == 'age':
            if value < 18: # change this value to whatever your country's minimum driving age is
                print('Sorry, you still cannot drive')
            else:
                print('Okay, you can drive now')
                # we don't need to be notified anymore, so we unsubscribe
                self.person.property_changed.remove(
                    self.person_changed
                )

Let's run our code:

In [4]:
p = Person()
ta = TrafficAuthority(p)
for age in range(15, 22):
    print(f'Setting age to {age}')
    p.age = age

Setting age to 15
Sorry, you still cannot drive
Setting age to 16
Sorry, you still cannot drive
Setting age to 17
Sorry, you still cannot drive
Setting age to 18
Okay, you can drive now
Setting age to 19
Setting age to 20
Setting age to 21


As you can see from the results above, as soon as the `Person` hits the minimum driving age, the `TrafficAuthority` unsubscribes from the age property change event and stops broadcasting age checks.